## HW 11 - Data fitting with various functions - Due Monday Nov 8, 2021


Dowload the data from the SDSS DR16:<br>
https://data.sdss.org/sas/dr16/apogee/spectro/aspcap/r12/l33/allStar-r12-l33.fits
NOTE: it is a large file and will take a while.


Using masking. Select the approprate fitting function and fit the following selections of the data:
1. Select stars only with with $-1< GLAT < 1$ <br>
   __Fit TEFF,  (TEFF\_ERR)  vs.  LogG (LOGG\_ERR)__<br>
<br>
2. Select stars only with with $GLAT < -10$ or $GLAT > 10$ <br>
   __Fit {\bf  Fe\_H,  (Fe\_H\_ERR)  vs.  O\_FE (O\_FE\_ERR)__<br>
<br>
3. Select stars only with with $-2< GLAT < 2$ <br>
   __Fit {\bf  Fe\_H,  (Fe\_H\_ERR)  vs.  O\_FE (O\_FE\_ERR)__<br>
<br>
4. Select stars only with with $GLAT < -10$ or $GLAT > 10$ <br>
   __Fit {\bf  TEFF,  (TEFF\_ERR)   vs.  (J - K) (J\_ERR, K\_ERR)__<br>
<br>
5. Select stars only with with $-10< GLAT < 10$<br>
   __Fit {\bf  GLON  vs.  VHEILO\_AVG (VERR)__<br>
<br>
6. __Fit {\bf NVISITS vs.  VSCATTER (VERR)__ For NVISITS $>= 1$<br>
<br>
7. __Fit {\bf  Fe\_H,  (Fe\_H\_ERR)  vs.  NI\_FE (NI\_FE\_ERR)__<br>
<br>

For each of the above selections:

1. What are the best-fit values of the parameters? 
2. Which fucntion fits the data best?
3. Is there a second population? (Can you fit outliers with a reasonable linear trend)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from astropy.io import fits 
import scipy.optimize as opt

# POTENTIAL FITTING FUNCTIONS
def linear(x,m,b):
    return m*x+b

def poly2(x,a,b,c):
    return a*(x**2)+(b*x)+c

def poly3(x,a,b,c,d):
    return (a*(x**3))+(b*(x**2))+(c*x)+d

def exponential(x,a,b,c,d):
    return a*np.exp(b*x + c) + d

def cosfit (x,a,b,c,d):
    return a*np.cos(b*x + c) + d

def logfit(x,a,b,c,d):
    return a*np.log(b*x + c) + d

def gausian(x,mu,sigma):
    part1 = (1.0/np.sqrt(2*np.pi*sigma**2))
    return (part1*(np.exp((-1*x - mu)**2/(2*sigma**2))))

def lorentzian_cauchy (x,a,b,c):
    return (c)*(b**2/((x-a)**2 + b**2))

def sigmoid(x,a,b,c,d):
    return (c/(1 + np.exp(-b*(x-a))) + d)

def mcFit2(func, x, y,  x_err,y_err, p0=[1,1]):
    slope = list()
    y_ints = list()
    iters = 100 
    for i in range(iters):
        # remember random normal distribution (Gaussian)
        weightsx = np.random.randn(len(y))
        weightsy = np.random.randn(len(y))

        y_adj = y + y_err*weightsy
        x_adj = x + x_err*weightsx 

        popt, pcov = opt.curve_fit(func, x_adj, y_adj, p0=p0)
        slope.append(popt[0])
        y_ints.append(popt[1])

    return (np.median(slope),np.median(y_ints))


def mcFit3(func, x, y, x_err,y_err, p0=[1,1,1]):
    paramA = list()
    paramB = list()
    paramC = list()
    iters = 100 
    for i in range(iters):
        # remember random normal distribution (Gaussian)
        weightsx = np.random.randn(len(y))
        weightsy = np.random.randn(len(y))

        y_adj = y + y_err*weightsy
        x_adj = x + x_err*weightsx 

        popt, pcov = opt.curve_fit(func, x_adj, y_adj, p0=p0)
        paramA.append(popt[0])
        paramB.append(popt[1])
        paramC.append(popt[2])
    return (np.median(paramA),np.median(paramB),np.median(paramC))

def mcFit4(func, x, y, x_err,y_err,  p0=[1,1,1,1]):
    paramA = list()
    paramB = list()
    paramC = list()
    paramD = list()
    iters = 100 
    for i in range(iters):
        # remember random normal distribution (Gaussian)
        weightsx = np.random.randn(len(y))
        weightsy = np.random.randn(len(y))

        y_adj = y + y_err*weightsy
        x_adj = x + x_err*weightsx 

        popt, pcov = opt.curve_fit(func, x_adj, y_adj, p0=p0)
        paramA.append(popt[0])
        paramB.append(popt[1])
        paramC.append(popt[2])
        paramD.append(popt[3])

    return (np.median(paramA),np.median(paramB),np.median(paramC),np.median(paramD))

# READ IN FITS FILES
star_hdus = fits.open('allStar-r12-l33.fits')
star = star_hdus[1].data
print(star_hdus[1].columns)
star_hdus.close()

# TWO BITWISE FLAGS FOR BAD DATA             
badbits = 2**23        # aspcapstar flag - Chemistry
suspectbits = 2**16    # star flag - Stellar parameters

# Make a Boolena Mask to remove bad data
gd = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) 
good = np.where(gd)[0]


# PLOT TO VERIFY FITS FILE READ IN & BAD MASK DATA
fig = plt.figure(figsize=(20,15))
ax = fig.add_subplot(111)
ax.scatter(star['RA'][good],star['DEC'][good],s=1,c='b',alpha=0.1)

ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
plt.show()




In [ ]:
# -1<GLAT<1 TEFF vs. LOGG

ct1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['GLAT'] < 1) & (star['GLAT'] > -1) & (star['TEFF'] > -9990) &  (star['LOGG'] > -9990)
cut1 = np.where(ct1)[0]

#LINEAR
slope, intercept = mcFit2(linear, star['TEFF'][cut1], star['LOGG'][cut1], star['TEFF_ERR'][cut1], star['LOGG_ERR'][cut1])
print("LINEAR: ",slope, intercept)

#POLY2
prA1, prB1, prC1 = mcFit3(poly2, star['TEFF'][cut1], star['LOGG'][cut1], star['TEFF_ERR'][cut1], star['LOGG_ERR'][cut1], p0=[0,0,0])
print("POLY2: ",prA1, prB1, prC1)

#POLY3
prA2, prB2, prC2, prD2 = mcFit4(poly3, star['TEFF'][cut1], star['LOGG'][cut1], star['TEFF_ERR'][cut1], star['LOGG_ERR'][cut1], p0=[0,0,0,0])
print("POLY3: ",prA2, prB2, prC2, prD2)

fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(111)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.scatter(star['TEFF'][cut1],star['LOGG'][cut1],s=1,c='b',alpha=0.1)
ax1.errorbar((star['TEFF'][cut1]),star['LOGG'][cut1], xerr=(star['TEFF_ERR'][cut1]), yerr=(star['LOGG_ERR'][cut1]), ecolor='grey',fmt='none', capsize=5, zorder=0)
x_plot = np.linspace(3000,10000,1000)

ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')
ax1.plot(x_plot,poly2(x_plot, prA1, prB1, prC1), label='poly2: FIT')
ax1.plot(x_plot,poly3(x_plot, prA2, prB2, prC2, prD2), label='poly3: BEST FIT')

plt.legend(loc='best', fontsize=18)
plt.show()

In [ ]:
# -10<GLAT<10 FE_H vs. O_FE

ct1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['Fe_H'] > -9990) &  (star['O_FE'] > -9990) & ((star['GLAT'] < 10) | (star['GLAT'] > -10)) 
cut1 = np.where(ct1)[0]

#LINEAR
slope, intercept = mcFit2(linear, star['Fe_H'][cut1], star['O_FE'][cut1], star['Fe_H_ERR'][cut1], star['O_FE_ERR'][cut1])
print("LINEAR: ",slope, intercept)

#POLY3
prA2, prB2, prC2, prD2 = mcFit4(poly3, star['Fe_H'][cut1], star['O_FE'][cut1], star['Fe_H_ERR'][cut1], star['O_FE_ERR'][cut1], p0=[0,0,0,0])
print("POLY3: ",prA2, prB2, prC2, prD2)

fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(111)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.scatter(star['Fe_H'][cut1],star['O_FE'][cut1],s=1,c='b',alpha=0.1)
ax1.errorbar((star['Fe_H'][cut1]),star['O_FE'][cut1], xerr=(star['Fe_H_ERR'][cut1]), yerr=(star['O_FE_ERR'][cut1]), ecolor='grey',fmt='none', capsize=5, zorder=0)
x_plot = np.linspace(-3,2,1000) 

# X-PLOTING FOR FITS
ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')
#ax1.plot(x_plot,poly2(x_plot, prA1, prB1, prC1), label='poly2: FIT')
ax1.plot(x_plot,poly3(x_plot, prA2, prB2, prC2, prD2), label='poly3: FIT')

plt.legend(loc='best', fontsize=18)
plt.show()

In [ ]:
# −2<𝐺𝐿𝐴𝑇<2 Fe_H vs. O_FE

badbits = 2**23        # aspcapstar flag - Chemistry
suspectbits = 2**16    # star flag - Stellar parameters

# Make a Boolena Mask to remove bad data - AND ANY OTHER NEEDE CUTS
ct1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['GLAT'] < 2) & (star['GLAT'] > -2)
cut1 = np.where(ct1)[0]


## LINEAR FIT
slope, intercept = mcFit2(linear, star['RA'][cut1], star['DEC'][cut1], star['RA'][cut1]/1000, star['RA'][cut1]/1000)
print("LINEAR: ",slope, intercept)

## COSFITS
prA, prB, prC, prD = mcFit4(cosfit, star['RA'][cut1], star['DEC'][cut1], star['RA'][cut1]/1000, star['RA'][cut1]/1000,p0=[60,0.02,0,0])
print("COSFIT: ",prA, prB, prC, prD)


fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(111)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)

# PLOT DATA AND ERROR BARS
ax1.scatter(star['RA'][cut1],star['DEC'][cut1],s=1,c='b',alpha=0.1)
ax1.errorbar((star['RA'][cut1]),star['DEC'][cut1], xerr=(star['RA'][cut1]/1000), yerr=(star['DEC'][cut1]/1000), ecolor='grey',fmt='none', capsize=5, zorder=0)

# PLOT FITS
x_plot = np.linspace(0,360,1000) # X-PLOTING FOR FITS
ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')
ax1.plot(x_plot,cosfit(x_plot, prA, prB, prC, prD), label='cosfit: FIT')

plt.legend(loc='best', fontsize=18)
plt.show()


In [ ]:
#𝐺𝐿𝐴𝑇<−10  or 𝐺𝐿𝐴𝑇>10 TEFF vs. (J - K)

ct1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['J'] > -9990) &  (star['K'] > -9990) & (star['TEFF'] > -9990) & ((star['GLAT'] < 10) | (star['GLAT'] > -10)) 
cut1 = np.where(ct1)[0]

ct4 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) & (np.bitwise_and(star['starflag'], suspectbits) == 0) &\

      (star['TEFF'] > -9000) & (star['J'] > -9000) & (star['K'] > -9000) & (star['J_ERR'] > -9000) &\

      (star['K_ERR'] > -9000) & (star['J_ERR'] < 1) & (star['K_ERR'] < 1) & ((star['GLAT'] > 10) | (star['GLAT'] < -10))

cut4 = np.where(ct4)[0]

#LINEAR
slope, intercept = mcFit2(linear, star['GLAT'][cut1], star['J'][cut1], np.sqrt(star['J_ERR'][cut4]**2 + star['K_ERR'][cut4]**2, star['J_ERR'][cut1])
print("LINEAR: ",slope, intercept)


fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(111)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.scatter(star['GLAT'][cut1],star['J'][cut1],s=1,c='b',alpha=0.1)

x_plot = np.linspace(-3,2,1000) 

# X-PLOTING FOR FITS
ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')


plt.legend(loc='best', fontsize=18)
plt.show()

In [ ]:
# −10<𝐺𝐿𝐴𝑇<10 GLON vs. VHEILO_AVG

ct1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['GLAT'] < 10) & (star['GLAT'] > -10) & (star['GLON'] > -9990) &  (star['VERR'] > -9990)
cut1 = np.where(ct1)[0]

#LINEAR
slope, intercept = mcFit2(linear, star['GLON'][cut1], star['VERR'][cut1], 0, 0)
print("LINEAR: ",slope, intercept)

#POLY2
prA1, prB1, prC1 = mcFit3(poly2, star['GLON'][cut1], star['VERR'][cut1], 0, 0, p0=[0,0,0])
print("POLY2: ",prA1, prB1, prC1)

#POLY3
prA2, prB2, prC2, prD2 = mcFit4(poly3, star['GLON'][cut1], star['VERR'][cut1], 0, 0, p0=[0,0,0,0])
print("POLY3: ",prA2, prB2, prC2, prD2)

fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(111)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.scatter(star['GLON'][cut1],star['VERR'][cut1],s=1,c='b',alpha=0.1)
ax1.errorbar((star['GLON'][cut1]),star['VERR'][cut1], xerr=(0), yerr=(0), ecolor='grey',fmt='none', capsize=5, zorder=0)
x_plot = np.linspace(-3,2,1000)

# X-PLOTING FOR FITS
ax1.plot(x_plot,linear(x_plot, slope, intercept), label='linear: FIT')
ax1.plot(x_plot,poly2(x_plot, prA1, prB1, prC1), label='poly2: FIT')
ax1.plot(x_plot,poly3(x_plot, prA2, prB2, prC2, prD2), label='poly3: FIT')

plt.legend(loc='best', fontsize=18)
plt.show()

In [ ]:
#Fit {\bf NVISITS vs. VSCATTER (VERR) For NVISITS  >=1 

ct1 = (np.bitwise_and(star['aspcapflag'], badbits) == 0) &\
     (np.bitwise_and(star['starflag'], suspectbits) == 0) &\
     (star['NVISITS'] >= 1) & (star['NVISITS'] > -9990) &  (star['VERR'] > -9990)
cut1 = np.where(ct1)[0]

fig1 = plt.figure(figsize=(20,15))
ax1 = fig1.add_subplot(111)
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.scatter(star['Fe_H'][cut1],star['O_FE'][cut1],s=1,c='b',alpha=0.1)
ax1.errorbar((star['Fe_H'][cut1]),star['O_FE'][cut1], xerr=(star['Fe_H_ERR'][cut1]), yerr=(star['O_FE_ERR'][cut1]), ecolor='grey',fmt='none', capsize=5, zorder=0)
x_plot = np.linspace(-10000,-9000,1000) 

#plt.legend(loc='best', fontsize=18)
plt.show()